### Libraries

In [1]:
%%capture
%reset -f                 # reset variables in the workspace
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)
import numpy as np
import datetime

'machine learning library imports'
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Load data

In [2]:
# Define the augmentation method and the data directory

AUGMENTATION = 'SMOTE'
data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])     

Loading complete.
Train data: 1500000 rows, 85 columns. 
Test data: 381934 rows, 85 columns.


### Data preparation

In [3]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [4]:
# Encode the training and test labels if needed
X_train_enc, X_test_enc = utils.encode_categorical(X_train, X_test)

No categorical features found. Returning original datasets.


#### Label encoding

In [5]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model training

In [6]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train_enc, random_state=42)

# Instantiate model
RF = RandomForestClassifier(random_state = 42)

# Train the model
RF_clf =  RF.fit(X_train, y_train)

### Model Evaluation

In [7]:
predictions = RF_clf.predict(X_test)

In [8]:
# Calculate metrics 
accuracy = metrics.accuracy_score(y_test_enc, predictions)
precision_w = metrics.precision_score(y_test_enc, predictions, average='weighted', zero_division=1)
recall_w = metrics.recall_score(y_test_enc, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test_enc, predictions, average='weighted')
precision_m = metrics.precision_score(y_test_enc, predictions, average='macro', zero_division=1)
recall_m = metrics.recall_score(y_test_enc, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test_enc, predictions, average='macro')

In [9]:
# Create dictionary for results
results = {
    "model": "Neural Net",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

utils.print_results_table(results)

╒══════════════════════╤═════════╕
│ Metric               │ Value   │
╞══════════════════════╪═════════╡
│ Accuracy             │ 98.08%  │
├──────────────────────┼─────────┤
│ Precision (macro)    │ 88.18%  │
├──────────────────────┼─────────┤
│ Recall (macro)       │ 91.78%  │
├──────────────────────┼─────────┤
│ F1 (macro)           │ 88.98%  │
├──────────────────────┼─────────┤
│ Precision (weighted) │ 98.18%  │
├──────────────────────┼─────────┤
│ Recall (weighted)    │ 98.08%  │
├──────────────────────┼─────────┤
│ F1 (weighted)        │ 98.12%  │
╘══════════════════════╧═════════╛


#### Save Metrics Results 

In [10]:
# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/randomforest.csv')

#### Confusion Matrix

In [12]:
conf_mat = metrics.confusion_matrix(y_test_enc, predictions)

# attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
# 'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
# 'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                           index = list(le.classes_), 
                           columns = list(le.classes_))
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'

# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4513,0,0,0,0,102,0,0,0,74,93,0,0,0,0
DDoS_HTTP,0,8498,0,0,0,0,0,0,0,0,0,0,0,181,949
DDoS_ICMP,0,0,13466,0,0,35,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,10009,0,0,0,0,0,0,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,3,0,0,0,0,112,0,0,0,22,9,0,0,0,0
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,0,272776,0,0,0,0,0,0,0
Password,0,2,0,0,0,0,0,0,8486,0,0,1003,617,0,0
